In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def update_parameters_with_gd(parameters, grads, learning_rate):

    L = len(parameters) // 2 # number of layers in the neural networks

    # Update rule for each parameter
    for l in range(L):
        ### START CODE HERE ### (approx. 2 lines)
        W = parameters["W" + str(l+1)]
        b = parameters["b" + str(l+1)]
        dW = grads['dW' + str(l+1)]
        db = grads['db' + str(l+1)]
        parameters["W" + str(l+1)] = W - learning_rate*dW
        parameters["b" + str(l+1)] = b - learning_rate*db
        ### END CODE HERE ###
        
    return parameters

In [5]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
   
    np.random.seed(seed)            # To make your "random" minibatches the same as ours
    m = X.shape[1]                  # number of training examples
    mini_batches = []
        
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1,m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X = shuffled_X[:,k*mini_batch_size:(k+1)*mini_batch_size]
        mini_batch_Y = shuffled_Y[:,k*mini_batch_size:(k+1)*mini_batch_size]
        ### END CODE HERE ###
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        ### START CODE HERE ### (approx. 2 lines)
        lastbeg = m//mini_batch_size*mini_batch_size
        mini_batch_X = shuffled_X[:,lastbeg:]
        mini_batch_Y = shuffled_Y[:,lastbeg:]
        ### END CODE HERE ###
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [6]:
def initialize_velocity(parameters):
    
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    
    # Initialize velocity
    for l in range(L):
        ### START CODE HERE ### (approx. 2 lines)
        W = parameters['W' + str(l+1)]
        b = parameters['b' + str(l+1)]
        v["dW" + str(l+1)] = np.zeros(W.shape)
        v["db" + str(l+1)] = np.zeros(b.shape)
        ### END CODE HERE ###
        
    return v

In [7]:
def update_parameters_with_momentum(parameters, grads, v, beta, learning_rate):

    L = len(parameters) // 2 # number of layers in the neural networks
    
    # Momentum update for each parameter
    for l in range(L):
        
        ### START CODE HERE ### (approx. 4 lines)
        W = parameters['W' + str(l+1)]
        b = parameters['b' + str(l+1)]
        vdW = v["dW" + str(l+1)]
        vdb = v["db" + str(l+1)]
        dW = grads['dW' + str(l+1)]
        db = grads['db' + str(l+1)]
        # compute velocities
        v["dW" + str(l+1)] = beta*vdW + (1-beta)*dW
        v["db" + str(l+1)] = beta*vdb + (1-beta)*db
        vdW = v["dW" + str(l+1)]
        vdb = v["db" + str(l+1)]
        # update parameters
        parameters["W" + str(l+1)] = W - learning_rate*vdW
        parameters["b" + str(l+1)] = b - learning_rate*vdb
        ### END CODE HERE ###
        
    return parameters, v

In [8]:
def initialize_adam(parameters) :
    
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    s = {}
    
    # Initialize v, s. Input: "parameters". Outputs: "v, s".
    for l in range(L):
    ### START CODE HERE ### (approx. 4 lines)
        W = parameters['W' + str(l+1)]
        b = parameters['b' + str(l+1)]    
        v["dW" + str(l+1)] = np.zeros(W.shape)
        v["db" + str(l+1)] = np.zeros(b.shape)
        s["dW" + str(l+1)] = np.zeros(W.shape)
        s["db" + str(l+1)] = np.zeros(b.shape)
    ### END CODE HERE ###
    
    return v, s

In [9]:
def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
    
    L = len(parameters) // 2                 # number of layers in the neural networks
    v_corrected = {}                         # Initializing first moment estimate, python dictionary
    s_corrected = {}                         # Initializing second moment estimate, python dictionary
    
    # Perform Adam update on all parameters
    for l in range(L):
        
        W = parameters['W' + str(l+1)]
        b = parameters['b' + str(l+1)]
        dW = grads['dW' + str(l+1)]
        db = grads['db' + str(l+1)]
        
        vdW = v["dW" + str(l+1)]
        vdb = v["db" + str(l+1)]
        sdW = s["dW" + str(l+1)]
        sdb = s["db" + str(l+1)]
        
        # Moving average of the gradients. Inputs: "v, grads, beta1". Output: "v".
        ### START CODE HERE ### (approx. 2 lines)
        vdW = beta1*vdW + (1-beta1)*dW
        vdb = beta1*vdb + (1-beta1)*db
        v["dW" + str(l+1)] = vdW
        v["db" + str(l+1)] = vdb       
        ### END CODE HERE ###

        # Compute bias-corrected first moment estimate. Inputs: "v, beta1, t". Output: "v_corrected".
        ### START CODE HERE ### (approx. 2 lines)
        vcdW = vdW / (1-beta1**t)
        vcdb = vdb / (1-beta1**t)
        v_corrected["dW" + str(l+1)] = vcdW
        v_corrected["db" + str(l+1)] = vcdb
        ### END CODE HERE ###

        # Moving average of the squared gradients. Inputs: "s, grads, beta2". Output: "s".
        ### START CODE HERE ### (approx. 2 lines)
        sdW = beta2*sdW + (1-beta2)*(dW**2)
        sdb = beta2*sdb + (1-beta2)*(db**2)
        s["dW" + str(l+1)] = sdW
        s["db" + str(l+1)] = sdb       
        ### END CODE HERE ###

        # Compute bias-corrected second raw moment estimate. Inputs: "s, beta2, t". Output: "s_corrected".
        ### START CODE HERE ### (approx. 2 lines)
        scdW = sdW / (1-beta2**t)
        scdb = sdb / (1-beta2**t)
        s_corrected["dW" + str(l+1)] = scdW
        s_corrected["db" + str(l+1)] = scdb
        ### END CODE HERE ###

        # Update parameters. Inputs: "parameters, learning_rate, v_corrected, s_corrected, epsilon". Output: "parameters".
        ### START CODE HERE ### (approx. 2 lines)
        parameters["W" + str(l+1)] = W-(learning_rate*
                                        (vcdW/(np.sqrt(scdW)+epsilon)))
        parameters["b" + str(l+1)] = b-(learning_rate*
                                        (vcdb/(np.sqrt(scdb)+epsilon)))
        ### END CODE HERE ###

    return parameters, v, s

In [10]:
def model(X, Y, layers_dims, optimizer, learning_rate = 0.0007, mini_batch_size = 64, beta = 0.9,
          beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8, num_epochs = 10000, print_cost = True):

    L = len(layers_dims)             # number of layers in the neural networks
    costs = []                       # to keep track of the cost
    t = 0                            # initializing the counter required for Adam update
    seed = 10                        # For grading purposes, so that your "random" minibatches are the same as ours
    
    # Initialize parameters
    parameters = initialize_parameters(layers_dims)

    # Initialize the optimizer
    if optimizer == "gd":
        pass # no initialization required for gradient descent
    elif optimizer == "momentum":
        v = initialize_velocity(parameters)
    elif optimizer == "adam":
        v, s = initialize_adam(parameters)
    
    # Optimization loop
    for i in range(num_epochs):
        
        # Define the random minibatches. We increment the seed to reshuffle differently the dataset after each epoch
        seed = seed + 1
        minibatches = random_mini_batches(X, Y, mini_batch_size, seed)

        for minibatch in minibatches:

            # Select a minibatch
            (minibatch_X, minibatch_Y) = minibatch

            # Forward propagation
            a3, caches = forward_propagation(minibatch_X, parameters)

            # Compute cost
            cost = compute_cost(a3, minibatch_Y)

            # Backward propagation
            grads = backward_propagation(minibatch_X, minibatch_Y, caches)

            # Update parameters
            if optimizer == "gd":
                parameters = update_parameters_with_gd(parameters, grads, learning_rate)
            elif optimizer == "momentum":
                parameters, v = update_parameters_with_momentum(parameters, grads, v, beta, learning_rate)
            elif optimizer == "adam":
                t = t + 1 # Adam counter
                parameters, v, s = update_parameters_with_adam(parameters, grads, v, s,
                                                               t, learning_rate, beta1, beta2,  epsilon)
        
        # Print the cost every 1000 epoch
        if print_cost and i % 1000 == 0:
            print ("Cost after epoch %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
                
    # plot the cost
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('epochs (per 100)')
    plt.title("Learning rate = " + str(learning_rate))
    plt.show()

    return parameters